In [1]:
from loaders.cognicity_loader import CognicityLoader
from learners.perceptron_learner import PerceptronLearner
from learners.svm_learner import SvmLearner

# import chennai_config
# config = chennai_config.config

import jakarta_config
config = jakarta_config.config

# import jakarta_only_pics_config
# config = jakarta_only_pics_config.config

# import jakarta_pic_and_text_config
# config = jakarta_pic_and_text_config.config

# import jakarta_relabeled_config
# config = jakarta_relabeled_config.config

RERUN = True

from image_recognition.goog_recog import GoogleLabeler
goog_learner = SvmLearner(config, CognicityLoader, GoogleLabeler)

from image_recognition.aws_recog import AwsLabeler
aws_learner = SvmLearner(config, CognicityLoader, AwsLabeler)

2019-08-22 17:11:56,737 - DEBUG - CognicityLoader constructed
2019-08-22 17:11:56,737 - DEBUG - GoogleLabeler constructed
2019-08-22 17:11:56,738 - DEBUG - CognicityLoader constructed
2019-08-22 17:11:56,738 - DEBUG - GoogleLabeler constructed
2019-08-22 17:11:56,788 - DEBUG - CognicityLoader constructed
2019-08-22 17:11:56,789 - DEBUG - AwsLabeler constructed
2019-08-22 17:11:56,790 - DEBUG - CognicityLoader constructed
2019-08-22 17:11:56,791 - DEBUG - AwsLabeler constructed


In [ ]:
len(config["flood_pkeys"])

In [ ]:
import random
validation_set = set(random.sample(config["flood_pkeys"], 10))
validation_set = validation_set.union(set(random.sample(config["no_flood_pkeys"], 10)))
len(validation_set)

In [ ]:
th, th0 = goog_learner.run_learner("goog_separator.p", rerun=True, validation_keys=validation_set, params={"T":1000, "print":True})
goog_learner.t_data_w_pkey[0,:]
val = goog_learner.val_data_w_pkey[0,:]
goog_learner.val_labels

In [ ]:
goog_learner.val_sd
goog_learner.t_sd.shape

In [ ]:
labs = goog_learner.labeler
len(labs.features)

In [ ]:
th, th0 = aws_learner.run_learner("aws_separator.p", rerun=RERUN, validation_keys=validation_set, params={"T":1000, "print":True})

In [ ]:
th.shape
aws_learner.t_sd.shape

In [ ]:
labs = aws_learner.labeler
len(labs.features)

In [ ]:
aws_labeler = aws_learner.labeler
a = aws_labeler.config["flood_pkeys"]
b = aws_labeler.config["no_flood_pkeys"]
c = a.union(b)
max(c)

In [ ]:
from nlp.bow_labeler import BowLabeler
# bow_learner = PerceptronLearner(config, CognicityLoader, BowLabeler)
bow_learner = SvmLearner(config, CognicityLoader, BowLabeler)
th, th0 = bow_learner.run_learner("bow_separator.p", rerun=RERUN, validation_keys=validation_set, params={"T":400, "print":True})

In [ ]:
bow_learner.t_sd.shape

In [2]:
from learners.ensemble_learner import EnsembleLearner
from learners.identity_learner import IdentityLearner
from flood_depth.flood_labeler import IdentityLabeler

from nlp.bow_labeler import BowLabeler
import random


results = dict()
for i in range(6, 11):
    validation_set = set(random.sample(config["flood_pkeys"], 40))
    validation_set = validation_set.union(set(random.sample(config["no_flood_pkeys"], 40)))
    
    goog_learner = SvmLearner(config, CognicityLoader, GoogleLabeler)
    th, th0 = goog_learner.run_learner("goog_separator.p", rerun=True, validation_keys=validation_set, params={"T":1000, "print":True})

    
    aws_learner = SvmLearner(config, CognicityLoader, AwsLabeler)
    th, th0 = aws_learner.run_learner("aws_separator.p", rerun=RERUN, validation_keys=validation_set, params={"T":1000, "print":True})
    bow_learner = SvmLearner(config, CognicityLoader, BowLabeler)
    th, th0 = bow_learner.run_learner("bow_separator.p", rerun=RERUN, validation_keys=validation_set, params={"T":400, "print":True})
    
    fh_learner = IdentityLearner(config, CognicityLoader, IdentityLabeler)
    meh = fh_learner.run_learner("iden_separator.p", rerun=RERUN, validation_keys=validation_set, params={"T":400, "print":True})
    
    learners = [ goog_learner, aws_learner, bow_learner, fh_learner]
    names = ["goog.p", "aws.p", "bow.p", "fh.p"]
    
    # learners = [ aws_learner, bow_learner]
    # names = ["aws.p", "bow.p"]
    en_learner = EnsembleLearner(config, names, learners)
    
    nn_model = en_learner.run_learner("jakarta_en.p", rerun=RERUN, validation_keys=validation_set, params={"hidden":i, "print":True, "T":2000})
    import math
    probs = math.e**en_learner.res
    probs
    import numpy as np
    p = probs.data.numpy()
    predicted = np.argmax(p, axis=1)
    val_labs = en_learner.val_labels
    true = np.where(val_labs <0, 0, 1)
    score = np.sum(predicted == true)/(true.shape[0])
    results[i] = (score, validation_set, en_learner, val_labs)
    print(score)
    print("COMPARE WITH SVM: ")
    t_data = en_learner.train_matrix[1:-1, :]
    t_labels = en_learner.train_matrix[-1, :]
    val_data = en_learner.val_matrix[1:-1, :]
    val_labels  = en_learner.val_matrix[-1, :]
    
    from sklearn import svm
    clf = svm.SVC(gamma="scale", kernel="rbf", degree=3)
            # sklearn expects rows to be data points, we've gone with columns
    clf.fit(t_data.T, t_labels)
    
    pred = clf.predict(val_data.T)
    
    correct = np.sum(val_labels == pred)
    
    total = val_data.shape[1]
    percent_correct = correct/total
    print("Num Correct " + str(correct) +
                     " Out of " + str(total))
    print("Val score: " + str(percent_correct))
    print("____________")
    

print(results)

2019-08-22 17:11:59,703 - DEBUG - CognicityLoader constructed
2019-08-22 17:11:59,703 - DEBUG - GoogleLabeler constructed
2019-08-22 17:11:59,704 - DEBUG - CognicityLoader constructed
2019-08-22 17:11:59,705 - DEBUG - GoogleLabeler constructed
2019-08-22 17:12:08,782 - INFO - Num Correct 47 Out of 79
2019-08-22 17:12:08,783 - INFO - Val score: 0.5949367088607594
2019-08-22 17:12:12,236 - DEBUG - CognicityLoader constructed
2019-08-22 17:12:12,237 - DEBUG - AwsLabeler constructed
2019-08-22 17:12:12,238 - DEBUG - CognicityLoader constructed
2019-08-22 17:12:12,239 - DEBUG - AwsLabeler constructed
2019-08-22 17:12:12,239 - DEBUG - logging from: default_jakarta_data/aws_labels_default.p
2019-08-22 17:12:28,365 - INFO - Num Correct 53 Out of 79
2019-08-22 17:12:28,366 - INFO - Val score: 0.6708860759493671
2019-08-22 17:12:35,905 - DEBUG - CognicityLoader constructed
2019-08-22 17:12:35,906 - DEBUG - CognicityLoader constructed
2019-08-22 17:13:09,933 - INFO - Num Correct 52 Out of 79
2019

                0
pkey             
1.0     -1.268525
3.0     -1.135426
4.0     -0.856265
9.0     -0.835979
10.0    -0.999707
20.0    -1.198872
23.0    -0.511189
24.0    -0.722396
101.0   -0.916940
116.0    0.053579
130.0   -0.978528
132.0   -1.164350
148.0    0.612065
160.0   -1.564207
162.0   -1.387846
163.0   -0.673752
168.0   -1.489927
208.0   -1.000010
209.0   -1.000197
210.0   -0.999615
211.0   -0.999983
212.0   -0.999925
213.0   -1.276650
280.0   -1.000061
281.0   -0.999882
282.0    0.232975
316.0   -1.048586
331.0   -0.433171
337.0   -1.000171
360.0   -0.786168
...           ...
36989.0  0.362459
36992.0 -1.387365
36996.0 -1.087414
37001.0 -1.160024
37013.0 -0.944959
37110.0 -0.999927
37121.0 -1.633733
37156.0 -1.000046
37180.0 -0.690822
37200.0 -1.018416
37229.0 -0.310272
37230.0 -1.157155
37254.0 -1.054043
37278.0 -1.053929
37302.0 -1.739258
37326.0 -1.166611
37338.0 -0.999852
37343.0 -1.710563
37344.0 -1.674851
37422.0 -0.206654
37446.0 -0.054821
37449.0 -0.825945
37450.0 -0

Epoch: 500
Loss: tensor(0.6077, grad_fn=<NllLossBackward>)
Epoch: 1000
Loss: tensor(0.5758, grad_fn=<NllLossBackward>)
Epoch: 1500
Loss: tensor(0.5499, grad_fn=<NllLossBackward>)
Epoch: 2000
Loss: tensor(0.5283, grad_fn=<NllLossBackward>)
Epoch: 2500
Loss: tensor(0.4630, grad_fn=<NllLossBackward>)
Epoch: 3000
Loss: tensor(0.3944, grad_fn=<NllLossBackward>)
Epoch: 3500
Loss: tensor(0.3433, grad_fn=<NllLossBackward>)
Epoch: 4000
Loss: tensor(0.3067, grad_fn=<NllLossBackward>)
Epoch: 4500
Loss: tensor(0.2808, grad_fn=<NllLossBackward>)
Epoch: 5000
Loss: tensor(0.2516, grad_fn=<NllLossBackward>)
Epoch: 5500
Loss: tensor(0.2377, grad_fn=<NllLossBackward>)
Epoch: 6000
Loss: tensor(0.2276, grad_fn=<NllLossBackward>)
Epoch: 6500
Loss: tensor(0.2199, grad_fn=<NllLossBackward>)
Epoch: 7000
Loss: tensor(0.2141, grad_fn=<NllLossBackward>)
Epoch: 7500
Loss: tensor(0.2095, grad_fn=<NllLossBackward>)
Epoch: 8000
Loss: tensor(0.2058, grad_fn=<NllLossBackward>)
Epoch: 8500
Loss: tensor(0.2029, grad_fn=

2019-08-22 17:13:52,514 - DEBUG - dumping model to: default_jakarta_data/jakarta_en.p
2019-08-22 17:13:52,575 - DEBUG - CognicityLoader constructed
2019-08-22 17:13:52,576 - DEBUG - GoogleLabeler constructed
2019-08-22 17:13:52,577 - DEBUG - CognicityLoader constructed
2019-08-22 17:13:52,577 - DEBUG - GoogleLabeler constructed


0.725
COMPARE WITH SVM: 
Num Correct 57 Out of 80
Val score: 0.7125
____________


...

2019-08-22 17:19:34,781 - INFO - Num Correct 50 Out of 77
2019-08-22 17:19:34,782 - INFO - Val score: 0.6493506493506493
2019-08-22 17:19:38,534 - DEBUG - CognicityLoader constructed
2019-08-22 17:19:38,535 - DEBUG - AwsLabeler constructed
2019-08-22 17:19:38,535 - DEBUG - CognicityLoader constructed
2019-08-22 17:19:38,536 - DEBUG - AwsLabeler constructed
2019-08-22 17:19:38,537 - DEBUG - logging from: default_jakarta_data/aws_labels_default.p
2019-08-22 17:19:56,484 - INFO - Num Correct 55 Out of 77
2019-08-22 17:19:56,485 - INFO - Val score: 0.7142857142857143
2019-08-22 17:20:03,816 - DEBUG - CognicityLoader constructed
2019-08-22 17:20:03,817 - DEBUG - CognicityLoader constructed
2019-08-22 17:20:37,635 - INFO - Num Correct 64 Out of 78
2019-08-22 17:20:37,636 - INFO - Val score: 0.8205128205128205
2019-08-22 17:21:01,664 - DEBUG - CognicityLoader constructed
2019-08-22 17:21:01,665 - DEBUG - IdentityLabeler constructed
2019-08-22 17:21:01,666 - DEBUG - CognicityLoader constructed

                0
pkey             
1.0     -1.282605
3.0     -1.111007
4.0     -0.849996
9.0     -0.635178
10.0    -0.999633
20.0    -1.128768
23.0    -0.531676
24.0    -0.717972
101.0   -0.933579
116.0   -0.088608
130.0   -0.973464
132.0   -1.166911
148.0    0.661200
160.0   -1.613677
162.0   -1.353926
163.0   -0.749583
168.0   -1.475529
208.0   -0.999951
209.0   -0.999977
210.0   -0.999969
211.0   -1.000102
212.0   -1.000331
213.0   -1.356318
280.0   -1.000258
281.0   -0.999559
282.0    0.288604
316.0   -1.096088
331.0   -0.519729
337.0   -0.999846
360.0   -0.824831
...           ...
36970.0 -0.796908
36989.0  0.164896
36992.0 -1.389208
36996.0 -1.074406
37001.0 -1.248870
37013.0 -0.940312
37110.0 -0.999916
37121.0 -1.702595
37156.0 -1.000190
37180.0 -0.622339
37200.0 -1.024975
37229.0 -0.322545
37230.0 -1.322480
37254.0 -1.256568
37278.0 -1.069315
37302.0 -1.851545
37326.0 -1.194476
37338.0 -1.000390
37343.0 -1.758257
37344.0 -1.784901
37422.0 -0.346689
37446.0  0.053903
37449.0 -0

Epoch: 500
Loss: tensor(0.6505, grad_fn=<NllLossBackward>)
Epoch: 1000
Loss: tensor(0.6387, grad_fn=<NllLossBackward>)
Epoch: 1500
Loss: tensor(0.6274, grad_fn=<NllLossBackward>)
Epoch: 2000
Loss: tensor(0.5825, grad_fn=<NllLossBackward>)
Epoch: 2500
Loss: tensor(0.5248, grad_fn=<NllLossBackward>)
Epoch: 3000
Loss: tensor(0.4752, grad_fn=<NllLossBackward>)
Epoch: 3500
Loss: tensor(0.4317, grad_fn=<NllLossBackward>)
Epoch: 4000
Loss: tensor(0.3952, grad_fn=<NllLossBackward>)
Epoch: 4500
Loss: tensor(0.3657, grad_fn=<NllLossBackward>)
Epoch: 5000
Loss: tensor(0.3425, grad_fn=<NllLossBackward>)
Epoch: 5500
Loss: tensor(0.3242, grad_fn=<NllLossBackward>)
Epoch: 6000
Loss: tensor(0.3098, grad_fn=<NllLossBackward>)
Epoch: 6500
Loss: tensor(0.2984, grad_fn=<NllLossBackward>)
Epoch: 7000
Loss: tensor(0.2895, grad_fn=<NllLossBackward>)
Epoch: 7500
Loss: tensor(0.2824, grad_fn=<NllLossBackward>)
Epoch: 8000
Loss: tensor(0.2768, grad_fn=<NllLossBackward>)
Epoch: 8500
Loss: tensor(0.2722, grad_fn=

2019-08-22 17:21:19,892 - DEBUG - dumping model to: default_jakarta_data/jakarta_en.p


0.75
COMPARE WITH SVM: 
Num Correct 61 Out of 80
Val score: 0.7625
____________
{8: (0.6875, {10368, 14851, 14852, 1923, 33411, 29703, 21383, 2055, 34826, 523, 33420, 4237, 6030, 5263, 35473, 9746, 9107, 30868, 917, 37013, 12434, 5271, 12566, 282, 6168, 15388, 16029, 4382, 23967, 11169, 19874, 31524, 6952, 13866, 34731, 10158, 21039, 29487, 22323, 23220, 15416, 22329, 25530, 23036, 23357, 23486, 2621, 13504, 27591, 15819, 6991, 6095, 36945, 18770, 34645, 29015, 22105, 13914, 3291, 21213, 14561, 14690, 27491, 10466, 23269, 24166, 28645, 6120, 13804, 32877, 33645, 23281, 31345, 8179, 12276, 16255, 23288, 18171, 23676, 29695}, <learners.ensemble_learner.EnsembleLearner object at 0x7eff80d2aa90>, array([-1.,  1.,  1., -1., -1.,  1.,  1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,  1.,
        1., -1.,  1., -1.,  1., -1., -1.,  1., -1., -1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,  1., -1., -1.,
       -1.

In [ ]:
l = validation_set
sorted(l)

In [ ]:
t_data = en_learner.train_matrix[1:-1, :]
t_labels = en_learner.train_matrix[-1, :]
val_data = en_learner.val_matrix[1:-1, :]
val_labels  = en_learner.val_matrix[-1, :]

from sklearn import svm
clf = svm.SVC(gamma="scale", kernel="rbf", degree=3)
        # sklearn expects rows to be data points, we've gone with columns
clf.fit(t_data.T, t_labels)

pred = clf.predict(val_data.T)

correct = np.sum(val_labels == pred)

total = val_data.shape[1]
percent_correct = correct/total
print("Num Correct " + str(correct) +
                 " Out of " + str(total))
print("Val score: " + str(percent_correct))
print(pred)
t_data[:,2:5]

In [ ]:
en_learner.val_matrix

In [ ]:
predicted

In [ ]:
import pytorch
a = pytorch.tensor(.4)

In [ ]:
predicted

validation_set

In [ ]:
predicted